# Arabic Named-Entity Recognition (NER) — Assignment

This notebook guides you through building an Arabic NER model using the ANERCorp dataset (`asas-ai/ANERCorp`). Fill in the TODO cells to complete the exercise.

- **Objective:** Train a token-classification model (NER) that labels tokens with entity tags (e.g., people, locations, organizations).
- **Dataset:** `asas-ai/ANERCorp` — contains tokenized Arabic text and tag sequences.
- **Typical Labels:** `B-PER`, `I-PER` (person), `B-LOC`, `I-LOC` (location), `B-ORG`, `I-ORG` (organization), and `O` (outside/no entity). Your code should extract the exact label set from the dataset and build `label_list`, `id2label`, and `label2id` mappings.
- **Key Steps (what you will implement):**
  1. Load the dataset and inspect samples.
  2. Convert the provided words into sentence groupings (use `.` `?` `!` as sentence delimiters) before tokenization so sentence boundaries are preserved.
  3. Tokenize with a pretrained Arabic tokenizer and align tokenized sub-words with original labels (use `-100` for tokens to ignore in loss).
  4. Prepare `tokenized_datasets` and data collator for dynamic padding.
  5. Configure and run model training using `AutoModelForTokenClassification` and `Trainer`.
  6. Evaluate using `seqeval` (report precision, recall, F1, and accuracy) and run inference with a pipeline.

- **Evaluation:** Use the `seqeval` metric (entity-level precision, recall, F1). When aligning predictions and labels, filter out `-100` entries so only real token labels are compared.

- **Deliverables:** Completed notebook with working cells for data loading, tokenization/label alignment, training, evaluation, and an inference example. Add short comments explaining choices (e.g., sentence-splitting strategy, tokenizer settings).

Good luck — implement each TODO in order and run the cells to verify output.

In [1]:
# TODO: Install the required packages for Arabic NER with transformers
# Required packages: transformers, datasets, seqeval, evaluate, accelerate
# Use pip install with -q flag to suppress output

!pip install transformers datasets seqeval evaluate accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00


In [2]:
# TODO: List the files in the current directory to explore the workspace
# Hint: Use a simple command to display directory contents

# YOUR CODE HERE
!ls

sample_data


In [3]:
# TODO: Load the ANERCorp dataset and extract label mappings
# Steps:
# 1. Import required libraries (datasets, numpy)
# 2. Load the "asas-ai/ANERCorp" dataset using load_dataset()
# 3. Inspect the dataset structure - print the splits and a sample entry
# 4. Extract unique tags from the training split
# 5. Create label_list (sorted), id2label, and label2id mappings

# YOUR CODE HERE
import datasets
import numpy as np

dataset = datasets.load_dataset("asas-ai/ANERCorp")
print(f"Dataset Split: {dataset}")
print(f"Sample Entry: {dataset['train'][0:5]}")

unique_tags = {tag for tag in dataset["train"]["tag"]}

label_list = sorted(unique_tags)
id2label = dict(enumerate(label_list))
label2id = {v: k for k, v in id2label.items()}

print(f"\nLabel List: {label_list}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-83c5047e14e689(…):   0%|          | 0.00/855k [00:00<?, ?B/s]

data/test-00000-of-00001-245173671c05c71(…):   0%|          | 0.00/175k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/125102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25008 [00:00<?, ? examples/s]

Dataset Split: DatasetDict({
    train: Dataset({
        features: ['word', 'tag'],
        num_rows: 125102
    })
    test: Dataset({
        features: ['word', 'tag'],
        num_rows: 25008
    })
})
Sample Entry: {'word': ['فرانكفورت', '(د', 'ب', 'أ)', 'أعلن'], 'tag': ['B-LOC', 'O', 'O', 'O', 'O']}

Label List: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PERS', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PERS', 'O']


In [5]:
# TODO: Verify the dataset was loaded correctly
# Print the dataframe or dataset summary to inspect the data structure

# YOUR CODE HERE
print(dataset["train"].to_pandas().describe())
print("-"*23)
print(dataset["test"].to_pandas().describe())

          word     tag
count   125102  125102
unique   29252       9
top          .       O
freq      4149  111921
-----------------------
         word    tag
count   25008  25008
unique   9075      9
top         .      O
freq      961  21633


In [26]:
# TODO: Load tokenizer and create tokenization function
# Steps:
# 1. Import AutoTokenizer from transformers
# 2. Set model_checkpoint to "aubmindlab/bert-base-arabertv02"
# 3. Load the tokenizer using AutoTokenizer.from_pretrained()
# 4. Create tokenize_and_align_labels function that:
#    - Tokenizes the input text (is_split_into_words=True)
#    - Maps tokens to their original words
#    - Handles special tokens by setting them to -100
#    - Aligns labels with sub-word tokens
#    - Returns tokenized inputs with labels
# 5. Important: Convert words to sentences using punctuation marks ".?!" as sentence delimiters
#    - This helps the model understand sentence boundaries
#    - Hint (suggested approach): group `examples['word']` into sentence lists using ".?!" as end markers, e.g.:
#        sentences = []
#        current = []
#        for w in examples['word']:
#            current.append(w)
#            if w in ['.', '?', '!'] or (len(w) > 0 and w[-1] in '.?!'):
#                sentences.append(current)
#                current = []
#        if current:
#            sentences.append(current)
#      Then align `examples['tag']` accordingly to these sentence groups before tokenization.
# 6. Apply the function to the entire dataset using dataset.map()

from transformers import AutoTokenizer

# YOUR CODE HERE
model_checkpoint = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def group_into_sentences(words, tags):
    sentences, sentence_tags = [], []
    current_words, current_tags = [], []
    for w, t in zip(words, tags):
        current_words.append(w)
        current_tags.append(t)
        if w in ['.', '?', '!'] or (len(w) > 0 and w[-1] in '.?!'):
            sentences.append(current_words)
            sentence_tags.append(current_tags)
            current_words, current_tags = [], []
    if current_words:
        sentences.append(current_words)
        sentence_tags.append(current_tags)
    return sentences, sentence_tags

train_sentences, train_tags = group_into_sentences(dataset['train']['word'], dataset['train']['tag'])
test_sentences, test_tags = group_into_sentences(dataset['test']['word'], dataset['test']['tag'])


def tokenize_and_align_labels(words_list, tags_list):
    all_inputs = []
    for words, tags in zip(words_list, tags_list):
        tokenized_inputs = tokenizer(words, is_split_into_words=True, truncation=True, padding=True)
        word_ids = tokenized_inputs.word_ids()
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(label2id[tags[word_idx]])
            else:
                aligned_labels.append(label2id[tags[word_idx]] if tags[word_idx].startswith("I-") else -100)
            previous_word_idx = word_idx
        tokenized_inputs["labels"] = aligned_labels
        all_inputs.append(tokenized_inputs)
    return all_inputs

tokenized_train = tokenize_and_align_labels(train_sentences, train_tags)
tokenized_test = tokenize_and_align_labels(test_sentences, test_tags)

In [23]:
# TODO: Define the compute_metrics function for model evaluation
# Steps:
# 1. Import evaluate and load "seqeval" metric
# 2. Create compute_metrics function that:
#    - Extracts predictions from model outputs using argmax
#    - Filters out -100 labels (special tokens and sub-words)
#    - Converts prediction and label IDs back to label names
#    - Computes seqeval metrics (precision, recall, f1, accuracy)
#    - Returns results as a dictionary

import evaluate
import numpy as np

# YOUR CODE HERE
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    # TODO: Implement metric computation
    # Hint: Use np.argmax, filter -100 labels, use seqeval.compute()

    logits, labels = p
    predictions = np.argmax(logits, axis=-1)

    true_predictions = []
    true_labels = []

    for pred, lab in zip(predictions, labels):
        pred_labels = []
        true_lab = []

        for p_id, l_id in zip(pred, lab):
            if l_id != -100:  
                pred_labels.append(id2label[p_id])
                true_lab.append(id2label[l_id])

        true_predictions.append(pred_labels)
        true_labels.append(true_lab)

    results = seqeval.compute( predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [24]:
# TODO: Load the model and configure training
# Steps:
# 1. Import AutoModelForTokenClassification, TrainingArguments, Trainer, and DataCollatorForTokenClassification
# 2. Load the model using AutoModelForTokenClassification.from_pretrained() with:
#    - model_checkpoint
#    - num_labels based on label_list length
#    - id2label and label2id mappings
# 3. Create TrainingArguments with:
#    - output directory "arabert-ner"
#    - evaluation_strategy="epoch"
#    - learning_rate=2e-5
#    - batch_size=16 (both train and eval)
#    - num_train_epochs=3
#    - weight_decay=0.01
# 4. Create a DataCollatorForTokenClassification for dynamic padding
# 5. Initialize the Trainer with model, args, datasets, data_collator, tokenizer, and compute_metrics
# 6. Call trainer.train() to start training

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

# YOUR CODE HERE
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id)

args = TrainingArguments(output_dir="arabert-ner", eval_strategy="epoch", learning_rate=2e-5, per_device_train_batch_size=16,
    per_device_eval_batch_size=16, num_train_epochs=3, weight_decay=0.01)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(model=model, args=args, train_dataset=datasets.Dataset.from_list(tokenized_train), eval_dataset=datasets.Dataset.from_list(tokenized_test),
    data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metrics)

trainer.train()

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1187262438.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=datasets.Dataset.from_list(tokenized_train), eval_dataset=datasets.Dataset.from_list(tokenized_test),
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.157914,0.823722,0.768928,0.795383,0.961267
2,0.113800,0.140585,0.812337,0.818381,0.815348,0.969612
3,0.113800,0.145919,0.825144,0.815755,0.820423,0.970281


TrainOutput(global_step=801, training_loss=0.0858816999323508, metrics={'train_runtime': 280.8078, 'train_samples_per_second': 45.533, 'train_steps_per_second': 2.852, 'total_flos': 592339052900340.0, 'train_loss': 0.0858816999323508, 'epoch': 3.0})

In [25]:
# TODO: Test the trained model with inference
# Steps:
# 1. Import pipeline from transformers
# 2. Create an NER pipeline using the trained model and tokenizer
# 3. Use aggregation_strategy="simple" to merge sub-tokens back into words
# 4. Test the pipeline with an Arabic text sample
# 5. Pretty print the results showing entity, label, and confidence score

from transformers import pipeline

# YOUR CODE HERE
ner_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

text = "أعلن المدير التنفيذي لشركة أبل تيم كوك عن افتتاح فرع جديد في الرياض."
results = ner_pipeline(text)

# Pretty print results
for entity in results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.2f}")

Device set to use cuda:0


Entity: أبل, Label: ORG, Score: 0.97
Entity: تيم كوك, Label: PERS, Score: 0.99
Entity: الرياض, Label: LOC, Score: 0.99
